In [1]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


2.3.2
1.3.1
GeForce RTX 2070 SUPER


device(type='cuda', index=0)

In [2]:
torch.set_num_threads(16)

In [3]:
torch.get_num_threads()

16

#### Load Data

In [4]:
df_train = pd.read_csv('input/train.csv', dtype=np.float32)
df_test = pd.read_csv('input/test.csv', dtype=np.float32)
print(df_train.shape, df_test.shape)


(810000, 230) (10000, 227)


In [17]:
layer_cols = [c for c in df_train.columns if 'layer_' in c]
fea_cols = [c for c in df_train.columns if c not in layer_cols]

len(fea_cols), len(layer_cols)

(226, 4)

In [18]:
X = df_train[fea_cols].values
y = list(df_train.index)
X_test = df_test[fea_cols].values

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1, n_jobs=-1)

neigh.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=1, p=2,
                     weights='uniform')

In [19]:
neigh.predict(X)

array([     0,      1,      2, ..., 809997, 809998, 809999])

In [38]:
pred = neigh.predict(X_test)

In [43]:
df_pred = pd.DataFrame(pred, columns=['pred'])

In [44]:
df_pred

,pred
0,668197
1,415599
2,394095
3,236347
4,728817
...,...
9995,278315
9996,211840
9997,131252
9998,55327


In [35]:
df_train[layer_cols]

,layer_1,layer_2,layer_3,layer_4
0,10.0,10.0,10.0,10.0
1,10.0,10.0,10.0,20.0
2,10.0,10.0,10.0,30.0
3,10.0,10.0,10.0,40.0
4,10.0,10.0,10.0,50.0
...,...,...,...,...
809995,300.0,300.0,300.0,260.0
809996,300.0,300.0,300.0,270.0
809997,300.0,300.0,300.0,280.0
809998,300.0,300.0,300.0,290.0


In [46]:
df_submit = pd.merge(df_pred, df_train[layer_cols], left_on='pred', right_on=df_train.index)

In [51]:
df_submit.index.name = 'id'

In [53]:
df_submit[layer_cols].to_csv('first.csv')